# TCGA-GBMLGG Analysis Demo

- **Cohort**: Focuses on the TCGA-GBMLGG dataset, a vital resource merging Glioblastoma Multiforme (GBM) and Lower-Grade Glioma (LGG).
- **Goal**: Perform histological subtype classification.
- **Prediction Target**: Predict whether a tumor is an `astrocytoma`, `oligodendroglioma`, or `oligoastrocytoma` based on its multi-omics profile.

**Data Source:** Broad Institute FireHose (http://firebrowse.org/?cohort=GBMLGG)

In [1]:
import pandas as pd
from pathlib import Path
root = Path("/home/vicente/Github/BioNeuralNet/GBMLGG")

mirna_raw = pd.read_csv(root/"GBMLGG.miRseq_RPKM_log2.txt", sep="\t",index_col=0,low_memory=False)                            
rna_raw = pd.read_csv(root / "GBMLGG.uncv2.mRNAseq_RSEM_normalized_log2.txt", sep="\t",index_col=0,low_memory=False)
meth_raw = pd.read_csv(root/"GBMLGG.meth.by_mean.data.txt", sep='\t',index_col=0,low_memory=False)
clinical_raw = pd.read_csv(root / "GBMLGG.clin.merged.picked.txt",sep="\t", index_col=0, low_memory=False)

# display shapes and first few rows-columns of each file
display(mirna_raw.iloc[:3,:5])
display(mirna_raw.shape)

display(rna_raw.iloc[:3,:5])
display(meth_raw.shape)

display(meth_raw.iloc[:3,:5])
display(meth_raw.shape)

display(clinical_raw.iloc[:3,:5])
display(clinical_raw.shape)

,TCGA-06-0675-11,TCGA-06-0678-11,TCGA-06-0680-11,TCGA-06-0681-11,TCGA-06-AABW-11
gene,,,,,
hsa-let-7a-1,12.847399,13.789578,13.603454,13.346797,13.545128
hsa-let-7a-2,13.850719,14.792970,14.597877,14.344260,14.554888
hsa-let-7a-3,12.873946,13.810832,13.611074,13.364372,13.583039


(548, 531)

,TCGA-02-0047-01,TCGA-02-0055-01,TCGA-02-2483-01,TCGA-02-2485-01,TCGA-02-2486-01
gene,,,,,
?|100133144,1.619742,NaN,1.559100,3.999567,2.475344
?|100134869,2.757258,3.972445,3.801138,3.902759,2.264506
?|10357,5.773564,4.972440,5.915141,6.520796,5.966629


(20115, 685)

,TCGA-06-0125-01,TCGA-06-0125-02,TCGA-06-0152-01,TCGA-06-0152-02,TCGA-06-0171-01
Hybridization REF,,,,,
Composite Element REF,Beta_Value,Beta_Value,Beta_Value,Beta_Value,Beta_Value
A1BG,0.438986043005,0.565094788162,0.461699906718,0.534127262606,0.455267108058
A1CF,0.681141812896,0.724487443757,0.601439733092,0.632221318323,0.691054589549


(20115, 685)

,tcga-06-6391,tcga-19-a6j4,tcga-cs-6665,tcga-cs-6670,tcga-db-a4xc
Hybridization REF,,,,,
Composite Element REF,value,value,value,value,value
years_to_birth,44,68,51,43,26
vital_status,1,1,0,0,0


(14, 1110)

## Data Processing Summary

1. **Transpose Data:** All raw data (miRNA, RNA, etc.) is flipped so rows represent patients and columns represent features.
2. **Standardize Patient IDs:** Patient IDs in all tables are cleaned to the 12-character TCGA format (e.g., `TCGA-AB-1234`) for matching.
3. **Handle Duplicates:** Duplicate patient rows are averaged in the omics data. The first entry is kept for duplicate patients in the clinical data.
4. **Find Common Patients:** The script identifies the list of patients that exist in *all* datasets.
5. **Subset Data:** All data tables are filtered down to *only* this common list of patients, ensuring alignment.
6. **Extract Target:** The `histological_type` column is pulled from the processed clinical data to be used as the prediction target (y-variable).

In [ ]:
mirna = mirna_raw.T
rna = rna_raw.T
meth = meth_raw.T
clinical = clinical_raw.T

print(f"miRNA (samples, features): {mirna.shape}")
print(f"RNA (samples, features): {rna.shape}")
print(f"Methylation (samples, features): {meth.shape}")
print(f"Clinical (samples, features): {clinical.shape}")

def trim_barcode(idx):
    return idx.to_series().str.slice(0, 12)

# standarized patient IDs across all files
meth.index = trim_barcode(meth.index)
rna.index = trim_barcode(rna.index)
mirna.index = trim_barcode(mirna.index)
clinical.index = clinical.index.str.upper()
clinical.index.name = "Patient_ID"

# convert all data to numeric, coercing errors to NaN
meth = meth.apply(pd.to_numeric, errors='coerce')
rna = rna.apply(pd.to_numeric, errors='coerce')
mirna = mirna.apply(pd.to_numeric, errors='coerce')

# for any duplicate columns in the omics data, we average their values
meth = meth.groupby(meth.index).mean()
rna = rna.groupby(rna.index).mean()
mirna = mirna.groupby(mirna.index).mean()

# for any duplicate rows in the clinical data, we keep the first occurrence
clinical = clinical[~clinical.index.duplicated(keep='first')]

print(f"\nMethylation shape: {meth.shape}")
print(f"RNA shape: {rna.shape}")
print(f"miRNA shape: {mirna.shape}")
print(f"Clinical shape: {clinical.shape}")

for df in [meth, rna, mirna]:
    df.columns = df.columns.str.replace(r"\?", "unknown_", regex=True)
    df.columns = df.columns.str.replace(r"\|", "_", regex=True)
    df.columns = df.columns.str.replace("-", "_", regex=False)
    df.columns = df.columns.str.replace(r"_+", "_", regex=True)
    df.columns = df.columns.str.strip("_")
    
    df.fillna(df.mean(), inplace=True)

# to see which pateints are common across all data files
common_patients = sorted(list(set(meth.index)&set(rna.index)&set(mirna.index)&set(clinical.index)))

print(f"\nFound: {len(common_patients)} patients across all data types.")

# subset to only common patients
meth_processed = meth.loc[common_patients]
rna_processed= rna.loc[common_patients]
mirna_processed = mirna.loc[common_patients]
clinical_processed = clinical.loc[common_patients]

# extract target labels from clinical data
targets = clinical_processed['histological_type']

miRNA (samples, features): (531, 548)
RNA (samples, features): (701, 18328)
Methylation (samples, features): (685, 20115)
Clinical (samples, features): (1110, 14)

Methylation shape: (658, 20115)
RNA shape: (681, 18328)
miRNA shape: (517, 548)
Clinical shape: (1110, 14)

Found: 511 patients across all data types.


In [4]:
display(mirna_processed.iloc[:3,:5])
display(mirna_processed.shape)

display(rna_processed.iloc[:3,:5])
display(rna_processed.shape)

display(meth_processed.iloc[:3,:5])
display(meth_processed.shape)

display(clinical_processed.iloc[:3,:5])
display(clinical_processed.shape)

display(targets.value_counts())

gene,hsa_let_7a_1,hsa_let_7a_2,hsa_let_7a_3,hsa_let_7b,hsa_let_7c
TCGA-CS-4938,12.622353,13.632728,12.651613,14.208930,14.376942
TCGA-CS-4941,11.809808,12.815815,11.820061,13.047853,11.955006
TCGA-CS-4942,11.113995,12.128618,11.165523,12.481790,11.858545


(511, 548)

gene,unknown_100133144,unknown_100134869,unknown_10357,unknown_10431,unknown_155060
TCGA-CS-4938,3.123352,4.507940,8.069184,9.724198,7.511790
TCGA-CS-4941,5.187819,4.404406,7.291745,8.608326,8.344526
TCGA-CS-4942,3.562316,3.462602,7.532460,9.279502,7.034985


(511, 18328)

Hybridization REF,Composite Element REF,A1BG,A1CF,A2BP1,A2LD1
TCGA-CS-4938,NaN,0.683179,0.776869,0.652055,0.919739
TCGA-CS-4941,NaN,0.521934,0.784401,0.563447,0.865717
TCGA-CS-4942,NaN,0.610067,0.828194,0.607771,0.875369


(511, 20115)

Hybridization REF,Composite Element REF,years_to_birth,vital_status,days_to_death,days_to_last_followup
Patient_ID,,,,,
TCGA-CS-4938,value,31,0,NaN,3574
TCGA-CS-4941,value,67,1,234,NaN
TCGA-CS-4942,value,44,1,1335,NaN


(511, 14)

histological_type
astrocytoma          193
oligodendroglioma    191
oligoastrocytoma     127
Name: count, dtype: int64

In [ ]:
import bioneuralnet as bnn

# drop unwanted columns from clinical data
clinical_processed.drop(columns=["Composite Element REF"], errors="ignore", inplace=True)

# we transform the methylation beta values to M-values and drop unwanted columns
meth_m = meth_processed.drop(columns=["Composite Element REF"], errors="ignore")

# convert beta values to M-values using bioneuralnet utility with small epsilon to avoid log(0)
meth_m = bnn.utils.beta_to_m(meth_m, eps=1e-6) 

# lastly we turn the target labels into numerical classes
mapping = {"astrocytoma": 0, "oligodendroglioma": 1, "oligoastrocytoma": 2}
target_labels = targets.map(mapping).to_frame(name="target")

# as a safety check we align the indices once more
X_meth = meth_m.loc[common_patients]
X_rna = rna_processed.loc[common_patients]
X_mirna = mirna_processed.loc[common_patients]
Y_labels = target_labels.loc[common_patients]
clinical_final = clinical_processed.loc[common_patients]

print(f"\nDNA_Methylation shape: {X_meth.shape}")
print(f"RNA shape: {X_rna.shape}")
print(f"miRNA shape: {X_mirna.shape}")
print(f"Clinical shape: {clinical_final.shape}")
print(Y_labels.value_counts())

2025-11-08 13:06:12,202 - bioneuralnet.utils.data - INFO - Starting Beta-to-M value conversion (shape: (511, 20114)). Epsilon: 1e-06
2025-11-08 13:06:13,301 - bioneuralnet.utils.data - INFO - Beta-to-M conversion complete.



Methylation shape: (511, 20114)
RNA shape: (511, 18328)
miRNA shape: (511, 548)
Clinical shape: (511, 13)
target
0         193
1         191
2         127
Name: count, dtype: int64


## Feature Selection Methodology

### Supported Methods and Interpretation

**BioNeuralNet** provides three techniques for feature selection, allowing for different views of the data's statistical profile:

- **Variance Thresholding:** Identifies features with the **highest overall variance** across all samples.

- **ANOVA F-test:** Pinpoints features that best **distinguish between the target classes** (KIRC, KIRP, and KICH).

- **Random Forest Importance:** Assesses **feature utility** based on its contribution to a predictive non-linear model.

### GBMLGG Cohort Selection Strategy

A dimensionality reduction step was essential for managing the high-feature-count omics data:

- **High-Feature Datasets:** Both DNA Methylation (20,114) and RNA (18,328) required significant feature reduction.

- **Filtering Process:** The **top 6,000 features** were initially extracted from the Methylation and RNA datasets using all three methods.

- **Final Set:** A consensus set was built by finding the intersection of features selected by the ANOVA F-test and Random Forest Importance, ensuring both statistical relevance and model-based utility.

- **Low-Feature Datasets:** The miRNA data (548 features) was passed through **without selection**, as its feature count was already manageable.

In [9]:
import bioneuralnet as bnn

# feature selection
meth_highvar = bnn.utils.select_top_k_variance(X_meth, k=6000)
rna_highvar = bnn.utils.select_top_k_variance(X_rna, k=6000)

meth_af = bnn.utils.top_anova_f_features(X_meth, Y_labels, max_features=6000)
rna_af = bnn.utils.top_anova_f_features(X_rna, Y_labels, max_features=6000)

meth_rf = bnn.utils.select_top_randomforest(X_meth, Y_labels, top_k=6000)
rna_rf = bnn.utils.select_top_randomforest(X_rna, Y_labels, top_k=6000)

meth_var_set = set(meth_highvar.columns)
meth_anova_set = set(meth_af.columns)
meth_rf_set = set(meth_rf.columns)

rna_var_set = set(rna_highvar.columns)
rna_anova_set = set(rna_af.columns)
rna_rf_set = set(rna_rf.columns)

meth_inter1 = list(meth_anova_set & meth_var_set)
meth_inter2 = list(meth_rf_set & meth_var_set)
meth_inter3 = list(meth_anova_set & meth_rf_set)
meth_all_three = list(meth_anova_set & meth_var_set & meth_rf_set)

rna_inter4 = list(rna_anova_set & rna_var_set)
rna_inter5 = list(rna_rf_set & rna_var_set)
rna_inter6 = list(rna_anova_set & rna_rf_set)
rna_all_three = list(rna_anova_set & rna_var_set & rna_rf_set)

2025-11-08 13:15:33,012 - bioneuralnet.utils.preprocess - INFO - [Inf]: Replaced 0 infinite values
2025-11-08 13:15:33,012 - bioneuralnet.utils.preprocess - INFO - [NaN]: Replaced 0 NaNs after median imputation
2025-11-08 13:15:33,012 - bioneuralnet.utils.preprocess - INFO - [Zero-Var]: 0 columns dropped due to zero variance
2025-11-08 13:15:33,085 - bioneuralnet.utils.preprocess - INFO - Selected top 6000 features by variance
2025-11-08 13:15:35,778 - bioneuralnet.utils.preprocess - INFO - [Inf]: Replaced 0 infinite values
2025-11-08 13:15:35,779 - bioneuralnet.utils.preprocess - INFO - [NaN]: Replaced 0 NaNs after median imputation
2025-11-08 13:15:35,779 - bioneuralnet.utils.preprocess - INFO - [Zero-Var]: 0 columns dropped due to zero variance
2025-11-08 13:15:35,843 - bioneuralnet.utils.preprocess - INFO - Selected top 6000 features by variance
2025-11-08 13:15:38,824 - bioneuralnet.utils.preprocess - INFO - [Inf]: Replaced 0 infinite values
2025-11-08 13:15:38,824 - bioneuralnet.

In [11]:
print("FROM THE 6000 Methylation feature selection:\n")
print(f"Anova-F & variance selection share: {len(meth_inter1)} features")
print(f"Random Forest & variance selection share: {len(meth_inter2)} features")
print(f"Anova-F & Random Forest share: {len(meth_inter3)} features")
print(f"All three methods agree on: {len(meth_all_three)} features")

FROM THE 6000 Methylation feature selection:

Anova-F & variance selection share: 2704 features
Random Forest & variance selection share: 1768 features
Anova-F & Random Forest share: 1823 features
All three methods agree on: 809 features


In [12]:
print("\nFROM THE 6000 RNA feature selection:\n")
print(f"Anova-F & variance selection share: {len(rna_inter4)} features")
print(f"Random Forest & variance selection share: {len(rna_inter5)} features")
print(f"Anova-F & Random Forest share: {len(rna_inter6)} features")
print(f"All three methods agree on: {len(rna_all_three)} features")


FROM THE 6000 RNA feature selection:

Anova-F & variance selection share: 2183 features
Random Forest & variance selection share: 1977 features
Anova-F & Random Forest share: 2127 features
All three methods agree on: 763 features


## Feature Selection Summary: ANOVA-RF Intersection

The chosen strategy for feature selection is based on the **overlap** between features identified by the **ANOVA F-test** and **Random Forest Importance**. This approach offers comprehensive filtering by balancing class-based relevance (ANOVA) with non-linear model importance (Random Forest). The resulting feature sets are considered the most robust for downstream analysis.

### Feature Overlap Results

The following table details the number of features resulting from the intersection of different selection methods for each omics data type.

| Omics Data Type | ANOVA-F & Variance | RF & Variance | ANOVA-F & Random Forest (Selected) | All Three Agree |
| :--- | :--- | :--- | :--- | :--- |
| **Methylation** | 2,704 features | 1,768 features | **1,823 features** | 809 features |
| **RNA** | 2,183 features | 1,977 features | **2,127 features** | 763 features |

In [13]:
X_meth_selected = X_meth[meth_inter3]
X_rna_selected = X_rna[rna_inter6]

print("\nFinal Shapes for Modeling")
print(f"Methylation (X1): {X_meth_selected.shape}")
print(f"RNA-Seq (X2): {X_rna_selected.shape}")
print(f"miRNA-Seq (X3): {X_mirna.shape}")
print(f"Labels (Y): {Y_labels.shape}")


Final Shapes for Modeling
Methylation (X1): (511, 1823)
RNA-Seq (X2): (511, 2127)
miRNA-Seq (X3): (511, 548)
Labels (Y): (511, 1)


## Data Availability

To facilitate rapid experimentation and reproduction of our results, the fully processed and feature-selected dataset used in this analysis has been made available directly within the package.

Users can load this dataset, bypassing all preceding data acquisition, preprocessing, and feature selection steps. This allows users to proceed immediately from this step.

In [ ]:
import bioneuralnet as bnn

tgca_lgg = bnn.datasets.DatasetLoader("lgg")
display(tgca_lgg.shape)

# The dataset is returned as a dictionary. We extract each file independetly based on the name( Key).
dna_meth = tgca_lgg.data["meth"]
rna = tgca_lgg.data["rna"]
mirna = tgca_lgg.data["mirna"]
clinical = tgca_lgg.data["clinical"]
target = tgca_lgg.data["target"]

{'mirna': (511, 548),
 'target': (511, 1),
 'clinical': (511, 13),
 'rna': (511, 2127),
 'meth': (511, 1823)}

In [ ]:
# BioNeuralNet provides a preprocessing function to handle clinical data
clinical = tgca_lgg.data["clinical"]

# For more details on the preprocessing functions, see `bioneuralnet.utils.preprocess``
clinical_preprocessed = bnn.utils.preprocess_clinical(
    clinical, 
    target, 
    top_k=7, 
    scale=False, 
    ignore_columns=[ "days_to_last_followup",  "years_to_birth", "days_to_death", "date_of_initial_pathologic_diagnosis"])

display(clinical_preprocessed.iloc[:3,:5])

2025-11-08 13:33:26,861 - bioneuralnet.utils.preprocess - INFO - [Inf]: Replaced 0 infinite values
2025-11-08 13:33:26,862 - bioneuralnet.utils.preprocess - INFO - [NaN]: Replaced 717 NaNs after median imputation
2025-11-08 13:33:26,862 - bioneuralnet.utils.preprocess - INFO - [Zero-Var]: 1 columns dropped due to zero variance
2025-11-08 13:33:26,944 - bioneuralnet.utils.preprocess - INFO - Selected top 7 features by RandomForest importance


,histological_type_oligodendroglioma,histological_type_oligoastrocytoma,karnofsky_performance_score,radiation_therapy_no,radiation_therapy_yes
Patient_ID,,,,,
TCGA-CS-4938,False,False,90.0,True,False
TCGA-CS-4941,False,False,90.0,False,True
TCGA-CS-4942,False,False,70.0,False,True


In [18]:
import pandas as pd

X_train_full = pd.concat([dna_meth, rna, mirna], axis=1)

print(f"Nan values in X_train_full: {X_train_full.isna().sum().sum()}")
X_train_full = X_train_full.dropna()
print(f"Nan value in X_train_full after dropping: {X_train_full.isna().sum().sum()}")

print(f"X_train_full shape: {X_train_full.shape}")
# building the graph using the similarity graph function with k=15
A_train = bnn.utils.gen_similarity_graph(X_train_full, k=15)

print(f"\nNetwork shape: {A_train.shape}")

Nan values in X_train_full: 0
Nan value in X_train_full after dropping: 0
X_train_full shape: (511, 4498)

Network shape: (4498, 4498)


## Reproducibility and Seeding

To ensure our experimental results are fully reproducible, a single global seed is set at the beginning of the analysis.

This utility function propagates the seed to all sources of randomness, including `random`, `numpy`, and `torch` (for both CPU and GPU).

Critically, it also configures the PyTorch cuDNN backend to use deterministic algorithms.

In [1]:
import bioneuralnet as bnn

SEED = 118
bnn.utils.set_seed(SEED)

2025-11-08 16:33:32,623 - bioneuralnet.utils.data - INFO - Setting global seed for reproducibility to: 118
2025-11-08 16:33:32,633 - bioneuralnet.utils.data - INFO - CUDA available. Applying seed to all GPU operations
2025-11-08 16:33:32,634 - bioneuralnet.utils.data - INFO - Seed setting complete


## Hyperparameter Tuning

- First step on our benchmark analysis is dedicated to **hyperparameter tuning** for:
    - **SAGE:** GraphSAGE (Graph Sample and Aggregate)
    - **GCN:** Graph Convolutional Network
    - **GAT:** Graph Attention Network
    
- We use `tune=True` and `repeat_num=5` within **DPMON** (our classification framework) to let the framework automatically search for the best architectural and training parameters.

- After the tuning phase, we will manually set the best-performing parameters for each model.
- We then use these optimal configurations in the final cross-validation to rigorously test each model performance and stability.

In [ ]:
from pathlib import Path

# We rename the target to phenotype so we keep it consistent with DPMON expectations.
target = target.rename(columns={"target": "phenotype"})

dpmon_sage_tune = bnn.downstream_task.DPMON(
    adjacency_matrix=A_train,
    omics_list=[dna_meth, rna, mirna],
    phenotype_data=target,
    clinical_data=clinical_preprocessed,
    model="SAGE",
    tune=True,
    repeat_num=5,
    gpu=True, cuda=0,
    output_dir=Path("/home/vicente/Github/BioNeuralNet/dpmon_tuning/GBM_SAGE"),
)

dpmon_sage_tune.run()

dpmon_gcn_tune = bnn.downstream_task.DPMON(
    adjacency_matrix=A_train,
    omics_list=[dna_meth, rna, mirna],
    phenotype_data=target,
    clinical_data=clinical_preprocessed,
    model="GCN",
    tune=True,
    repeat_num=5,
    gpu=True, cuda=0,
    output_dir=Path("/home/vicente/Github/BioNeuralNet/dpmon_tuning/GBM_GCN"),
)

dpmon_gcn_tune.run()


dpmon_gat_tune = bnn.downstream_task.DPMON(
    adjacency_matrix=A_train,
    omics_list=[dna_meth, rna, mirna],
    phenotype_data=target,
    clinical_data=clinical_preprocessed,
    model="GAT",
    tune=True,
    repeat_num=5,
    gpu=True, cuda=0,
    output_dir=Path("/home/vicente/Github/BioNeuralNet/dpmon_tuning/GBM_GAT"),
)

dpmon_gcn_tune.run()

# Due to the extensive length, the terminal output has been omitted. To see the full terminal output for the results below please visit: https://github.com/UCD-BDLab/BioNeuralNet/TCGA-Notebooks/TCGA-GBMLGG.ipynb

## Analysis of Hyperparameter Optimization

**Hyperparameter tuning using `SEED = 118`**

### Tuning Search Space

**DPMON** parameter search space is extensive, offering over 12,000 discrete combinations plus a continuous space for learning rates. By sampling **50 trials** from this space, we are effectively probing it to find the distinct, optimal architecture best suited for each GNN (SAGE, GCN, and GAT).


| Parameter | SAGE (GraphSAGE) | GCN (Graph Convolutional) | GAT (Graph Attention) |
| :--- | :--- | :--- | :--- |
| **gnn\_layer\_num** | 2 | 64 | 16 |
| **gnn\_hidden\_dim** | 128 | 16 | 4 |
| **lr (Learning Rate)** | 0.005197 | 0.004397 | 0.003226 |
| **weight\_decay** | 0.046079 | 0.000405 | 0.046863 |
| **nn\_hidden\_dim1** | 1 | 64 | 16 |
| **nn\_hidden\_dim2** | 32 | 8 | 128 |
| **num\_epochs** | 4096 | 1024 | 512 |

### SAGE Model Configuration
The optimal SAGE configuration was **shallow but wide**, requiring an unusually long training time. This completely reverses the previous "deep and narrow" result, indicating that a high-dimensional feature representation was prioritized over recursive aggregation in this specific run.

### GCN Model Configuration
The GCN model favored a highly atypical architecture: **deep yet narrow**. The success of this extremely deep GCN suggests the model learned highly specialized, low-dimensional features over many layers, despite the typical risk of over-smoothing.

### GAT Model Configuration
The GAT model found a balanced, efficient architecture: **moderate depth and narrow**. It was the fastest to converge, suggesting GAT's attention mechanism effectively selected features, allowing it to perform well with a minimal, low-dimensional representation.

---

## Cross-Validation Results with 5 folds

| Model | Avg. Accuracy | Avg. F1 Weighted | Avg. F1 Macro |
| :--- | :--- | :--- | :--- |
| **GAT** | **0.9213** +/- 0.1321 | **0.9045** +/- 0.1656 | **0.8906** +/- 0.1910 |
| **GCN** | 0.8821 +/- 0.1353 | 0.8639 +/- 0.1666 | 0.8644 +/- 0.1620 |
| **SAGE** | 0.7567 +/- 0.1846 | 0.7311 +/- 0.2160 | 0.7135 +/- 0.2361 |

The cross-validation results demonstrate that the optimal architectures found with `SEED = 118` performed best with GAT, followed by GCN, and then SAGE.

### 1. GAT Performance

The GAT model achieved the highest average accuracy and F1 scores, suggesting its attention mechanism effectively utilized the narrow-but-moderately-deep architecture (16 layers, 4 hidden dim) for classification.

- Avg. Accuracy: 0.9213 (+/- 0.1321)
- Avg. F1 Weighted: 0.9045 (+/- 0.1656)

### 2. GCN Performance

GCN performed very strongly, showing that its highly unusual deep-and-narrow architecture (64 layers, 16 hidden dim) was successful. However, like the other models, it exhibited high standard deviations, indicating its performance was also unstable across the folds.

- Avg. Accuracy: 0.8821 (+/- 0.1353)
- Avg. F1 Weighted: 0.8639 (+/- 0.1666)

### 3. SAGE Performance

SAGE yielded the lowest average performance and the highest variability (largest +/- values). This suggests that the shallow-but-wide architecture (2 layers, 128 hidden dim) was less stable and less effective for this specific classification task compared to the other two models.

- Avg. Accuracy: 0.7567 (+/- 0.1846)
- Avg. F1 Weighted: 0.7311 (+/- 0.2160)


### Summary

While the GAT and GCN models achieved high average performance, all three models suffered from high standard deviations. This indicates that while the architectures are capable of high accuracy, their performance is highly sensitive.

**These results are shown below**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
from bioneuralnet.downstream_task import DPMON

X_meth_full = dna_meth
X_rna_full = rna
X_mirna_full = mirna
C_full = clinical_preprocessed
Y_full = target
A_full = A_train

patient_indices = np.arange(len(Y_full))
y_target_classes = Y_full.squeeze()

sage_params = {
    'layer_num': 2,
    'gnn_hidden_dim': 128,
    'lr': 0.005197038211089667,
    'weight_decay': 0.04607928341516355,
    'nn_hidden_dim1': 16,
    'nn_hidden_dim2': 32,
    'num_epochs': 4096
}

N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED) 
output_dir_base_sage = Path("/home/vicente/Github/BioNeuralNet/dpmon_cv_results_SAGE_FINAL")

accuracy_scores_sage = []
f1_macro_scores_sage = []
f1_weighted_scores_sage = []

for fold_num, (train_index, test_index) in enumerate(skf.split(patient_indices, y_target_classes)):
    print(f"\nRunning DPMON SAGE Fold {fold_num + 1}/{N_FOLDS}")

    X_train_omics = [X_meth_full.iloc[train_index], X_rna_full.iloc[train_index], X_mirna_full.iloc[train_index]]
    Y_train = Y_full.iloc[train_index]
    C_train = C_full.iloc[train_index]
    A_train_fold = A_full.iloc[train_index, train_index] 
    
    dpmon_fold = DPMON(
        adjacency_matrix=A_train_fold, omics_list=X_train_omics, phenotype_data=Y_train,
        clinical_data=C_train, repeat_num=1, tune=False, gpu=True, cuda=0,
        output_dir=output_dir_base_sage / f"fold_{fold_num + 1}",
        **sage_params
    )

    predictions_df, _ = dpmon_fold.run() 
    actual = predictions_df["Actual"]
    pred = predictions_df["Predicted"]

    test_f1_macro = f1_score(actual, pred, average='macro')
    test_f1_weighted = f1_score(actual, pred, average='weighted')
    test_acc = accuracy_score(actual, pred)

    accuracy_scores_sage.append(test_acc)
    f1_macro_scores_sage.append(test_f1_macro)
    f1_weighted_scores_sage.append(test_f1_weighted)
    
    print(f"SAGE Fold {fold_num + 1} | Accuracy: {test_acc:.4f} | F1-Macro: {test_f1_macro:.4f}")

mean_acc_sage = np.mean(accuracy_scores_sage)
std_acc_sage = np.std(accuracy_scores_sage)

mean_f1_macro_sage = np.mean(f1_macro_scores_sage)
std_f1_macro_sage = np.std(f1_macro_scores_sage)

mean_f1_weighted_sage = np.mean(f1_weighted_scores_sage)
std_f1_weighted_sage = np.std(f1_weighted_scores_sage)

print("\nClassification with SAGE and 5-FOLD Cross-Validation")
print(f"Avg. Accuracy: {mean_acc_sage:.4f} +/- {std_acc_sage:.4f}")
print(f"Avg. F1 Weighted: {mean_f1_weighted_sage:.4f} +/- {std_f1_weighted_sage:.4f}")
print(f"Avg. F1 Macro: {mean_f1_macro_sage:.4f} +/- {std_f1_macro_sage:.4f}")

# Due to the extensive length, the terminal output has been omitted. To see the full terminal output for the results below please visit: https://github.com/UCD-BDLab/BioNeuralNet/TCGA-Notebooks/TCGA-GBMLGG.ipynb

In [ ]:
output_dir_base_gcn = Path("/home/vicente/Github/BioNeuralNet/dpmon_cv_results_GCN_FINAL")

gcn_params = {
    'layer_num': 64,
    'gnn_hidden_dim': 16,
    'lr': 0.004397636847528492,
    'weight_decay': 0.00040569459559797747,
    'nn_hidden_dim1': 4,
    'nn_hidden_dim2': 8,
    'num_epochs': 1024
}

skf_gcn = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED) 

accuracy_scores_gcn = []
f1_macro_scores_gcn = []
f1_weighted_scores_gcn = []

for fold_num, (train_index, test_index) in enumerate(skf_gcn.split(patient_indices, y_target_classes)):
    print(f"\nDPMON GCN Fold {fold_num + 1}/{N_FOLDS}")

    X_train_omics = [X_meth_full.iloc[train_index], X_rna_full.iloc[train_index], X_mirna_full.iloc[train_index]]
    Y_train = Y_full.iloc[train_index]
    C_train = C_full.iloc[train_index]
    A_train_fold = A_full.iloc[train_index, train_index] 
    
    dpmon_fold = DPMON(
        adjacency_matrix=A_train_fold, omics_list=X_train_omics, phenotype_data=Y_train,
        clinical_data=C_train, repeat_num=1, tune=False, gpu=True, cuda=0,
        output_dir=output_dir_base_gcn / f"fold_{fold_num + 1}",
        **gcn_params
    )

    predictions_df, _ = dpmon_fold.run() 
    actual = predictions_df["Actual"]
    pred = predictions_df["Predicted"]

    test_acc = accuracy_score(actual, pred)
    test_f1 = f1_score(actual, pred, average='macro')
    test_f1w = f1_score(actual, pred, average='weighted')

    accuracy_scores_gcn.append(test_acc)
    f1_macro_scores_gcn.append(test_f1)
    f1_weighted_scores_gcn.append(test_f1w)
    
    print(f"GCN Fold {fold_num + 1} | Accuracy: {test_acc:.4f} | F1-Macro: {test_f1:.4f}")

mean_acc_gcn = np.mean(accuracy_scores_gcn)
std_acc_gcn = np.std(accuracy_scores_gcn)

mean_f1_macro_gcn = np.mean(f1_macro_scores_gcn)
std_f1_macro_gcn = np.std(f1_macro_scores_gcn)

mean_f1_weighted_gcn = np.mean(f1_weighted_scores_gcn)
std_f1_weighted_gcn = np.std(f1_weighted_scores_gcn)

print("Classification with GCN and 5-FOLD Cross-Validation")
print(f"Avg. Accuracy: {mean_acc_gcn:.4f} +/- {std_acc_gcn:.4f}")
print(f"Avg. F1 Weighted: {mean_f1_weighted_gcn:.4f} +/- {std_f1_weighted_gcn:.4f}")
print(f"Avg. F1 Macro: {mean_f1_macro_gcn:.4f} +/- {std_f1_macro_gcn:.4f}")

# Due to the extensive length, the terminal output has been omitted. To see the full terminal output for the results below please visit: https://github.com/UCD-BDLab/BioNeuralNet/TCGA-Notebooks/TCGA-GBMLGG.ipynb

In [ ]:
output_dir_base_gat = Path("/home/vicente/Github/BioNeuralNet/dpmon_cv_results_GAT_FINAL")

gat_params = {
    'layer_num': 16,
    'gnn_hidden_dim': 4,
    'lr': 0.003226646486974539,
    'weight_decay': 0.04686394640851084,
    'nn_hidden_dim1': 16,
    'nn_hidden_dim2': 128,
    'num_epochs': 512
}

skf_gat = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED) 

accuracy_scores_gat = []
f1_macro_scores_gat = []
f1_weighted_scores_gat = []

for fold_num, (train_index, test_index) in enumerate(skf_gat.split(patient_indices, y_target_classes)):
    print(f"\nDPMON GAT Fold {fold_num + 1}/{N_FOLDS}")

    X_train_omics = [X_meth_full.iloc[train_index], X_rna_full.iloc[train_index], X_mirna_full.iloc[train_index]]
    Y_train = Y_full.iloc[train_index]
    C_train = C_full.iloc[train_index]
    A_train_fold = A_full.iloc[train_index, train_index] 
    
    dpmon_fold = DPMON(
        adjacency_matrix=A_train_fold, omics_list=X_train_omics, phenotype_data=Y_train,
        clinical_data=C_train, repeat_num=1, tune=False, gpu=True, cuda=0,
        output_dir=output_dir_base_gat / f"fold_{fold_num + 1}",
        **gat_params
    )

    predictions_df, _ = dpmon_fold.run() 
    actual = predictions_df["Actual"]
    pred = predictions_df["Predicted"]

    test_acc = accuracy_score(actual, pred)
    test_f1 = f1_score(actual, pred, average='macro')
    test_f1w = f1_score(actual, pred, average='weighted')

    accuracy_scores_gat.append(test_acc)
    f1_macro_scores_gat.append(test_f1)
    f1_weighted_scores_gat.append(test_f1w)
    
    print(f"GAT Fold {fold_num + 1} | Accuracy: {test_acc:.4f} | F1-Macro: {test_f1:.4f}")

mean_acc_gat = np.mean(accuracy_scores_gat)
std_acc_gat = np.std(accuracy_scores_gat)

mean_f1_macro_gat = np.mean(f1_macro_scores_gat)
std_f1_macro_gat = np.std(f1_macro_scores_gat)

mean_f1_weighted_gat = np.mean(f1_weighted_scores_gat)
std_f1_weighted_gat = np.std(f1_weighted_scores_gat)

print("Classification with GAT and 5-FOLD Cross-Validation")
print(f"Avg. Accuracy: {mean_acc_gat:.4f} +/- {std_acc_gat:.4f}")
print(f"Avg. F1 Weighted: {mean_f1_weighted_gat:.4f} +/- {std_f1_weighted_gat:.4f}")
print(f"Avg. F1 Macro: {mean_f1_macro_gat:.4f} +/- {std_f1_macro_gat:.4f}")

# Due to the extensive length, the terminal output has been omitted. To see the full terminal output for the results below please visit: https://github.com/UCD-BDLab/BioNeuralNet/TCGA-Notebooks/TCGA-GBMLGG.ipynb